# Chapter 9: Optimizations Built-in

Performance is not an afterthought in Next.js—it's built into the framework's DNA. Next.js provides a suite of powerful optimization features that work out of the box to ensure your applications are fast, efficient, and provide excellent user experiences.

By the end of this chapter, you'll master Next.js's built-in optimization features, understand how to implement them effectively, and be able to build applications that load instantly and perform exceptionally well.

## 9.1 Image Optimization with next/image

Images are often the largest assets on a webpage, significantly impacting load times and user experience. Next.js's Image component provides automatic optimization, responsive images, lazy loading, and modern format support.

### Basic Image Usage

The simplest way to optimize images in Next.js:

```tsx
// components/OptimizedImage.tsx
import Image from 'next/image';

interface OptimizedImageProps {
  src: string;
  alt: string;
  width?: number;
  height?: number;
  priority?: boolean;
}

export function OptimizedImage({
  src,
  alt,
  width = 800,
  height = 600,
  priority = false,
}: OptimizedImageProps) {
  return (
    <div className="relative w-full aspect-video">
      <Image
        src={src}
        alt={alt}
        fill
        priority={priority}
        className="object-cover rounded-lg"
        sizes="(max-width: 768px) 100vw, (max-width: 1200px) 50vw, 33vw"
      />
    </div>
  );
}
```

### Responsive Images with Sizes

The `sizes` attribute helps browsers select the appropriate image size:

```tsx
// components/ResponsiveImageGallery.tsx
import Image from 'next/image';

interface GalleryImage {
  src: string;
  alt: string;
  width: number;
  height: number;
}

export function ResponsiveImageGallery() {
  const images: GalleryImage[] = [
    { src: '/images/gallery-1.jpg', alt: 'Gallery image 1', width: 1200, height: 800 },
    { src: '/images/gallery-2.jpg', alt: 'Gallery image 2', width: 1200, height: 800 },
    { src: '/images/gallery-3.jpg', alt: 'Gallery image 3', width: 1200, height: 800 },
    { src: '/images/gallery-4.jpg', alt: 'Gallery image 4', width: 1200, height: 800 },
    { src: '/images/gallery-5.jpg', alt: 'Gallery image 5', width: 1200, height: 800 },
    { src: '/images/gallery-6.jpg', alt: 'Gallery image 6', width: 1200, height: 800 },
  ];

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Image Gallery</h1>

      <div className="grid grid-cols-1 sm:grid-cols-2 lg:grid-cols-3 gap-6">
        {images.map((image, index) => (
          <div 
            key={index}
            className="relative aspect-[4/3] bg-gray-100 rounded-lg overflow-hidden"
          >
            <Image
              src={image.src}
              alt={image.alt}
              fill
              // Different sizes for different breakpoints
              sizes="(max-width: 640px) 100vw, (max-width: 1024px) 50vw, 33vw"
              className="object-cover hover:scale-105 transition-transform duration-300"
              // Lazy load all but the first image
              loading={index === 0 ? 'eager' : 'lazy'}
            />
          </div>
        ))}
      </div>
    </div>
  );
}
```

### Image Quality and Formats

Configure image quality and formats for optimal performance:

```tsx
// app/products/[id]/page.tsx
import Image from 'next/image';

interface Product {
  id: string;
  name: string;
  description: string;
  price: number;
  images: {
    thumbnail: string;
    main: string;
    gallery: string[];
  };
}

async function getProduct(id: string): Promise<Product> {
  // Fetch product data
  const response = await fetch(`https://api.example.com/products/${id}`);
  return response.json();
}

export default async function ProductPage({ params }: { params: { id: string } }) {
  const product = await getProduct(params.id);

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="grid grid-cols-1 lg:grid-cols-2 gap-8">
        {/* Main product image */}
        <div className="space-y-4">
          <div className="relative aspect-[4/3] bg-gray-100 rounded-lg overflow-hidden">
            <Image
              src={product.images.main}
              alt={product.name}
              fill
              priority // Load main image immediately
              quality={90} // Higher quality for main image
              sizes="(max-width: 1024px) 100vw, 50vw"
              className="object-cover"
            />
          </div>

          {/* Thumbnail gallery */}
          <div className="grid grid-cols-4 gap-4">
            {product.images.gallery.map((image, index) => (
              <div 
                key={index}
                className="relative aspect-square bg-gray-100 rounded overflow-hidden cursor-pointer hover:ring-2 hover:ring-blue-500 transition-all"
              >
                <Image
                  src={image}
                  alt={`${product.name} view ${index + 1}`}
                  fill
                  quality={75} // Lower quality for thumbnails
                  sizes="(max-width: 640px) 25vw, 100px"
                  className="object-cover"
                />
              </div>
            ))}
          </div>
        </div>

        {/* Product details */}
        <div>
          <h1 className="text-3xl font-bold text-gray-900 mb-4">
            {product.name}
          </h1>
          <p className="text-gray-600 mb-6">{product.description}</p>
          <p className="text-3xl font-bold text-blue-600 mb-6">
            ${product.price.toFixed(2)}
          </p>
          <button className="w-full bg-blue-600 text-white py-3 rounded-lg hover:bg-blue-700 transition-colors">
            Add to Cart
          </button>
        </div>
      </div>
    </div>
  );
}
```

### Image Placeholders and Blur Effects

Add blur-up placeholders for better perceived performance:

```tsx
// components/BlurUpImage.tsx
import Image from 'next/image';

interface BlurUpImageProps {
  src: string;
  alt: string;
  width: number;
  height: number;
}

export function BlurUpImage({ src, alt, width, height }: BlurUpImageProps) {
  return (
    <div className="relative bg-gray-200 rounded-lg overflow-hidden">
      <Image
        src={src}
        alt={alt}
        width={width}
        height={height}
        placeholder="blur"
        blurDataURL=""
        className="hover:scale-105 transition-transform duration-300"
      />
    </div>
  );
}
```

### Dynamic Image Optimization

Optimize images from external sources:

```tsx
// app/blog/[slug]/page.tsx
import Image from 'next/image';

interface BlogPost {
  id: string;
  slug: string;
  title: string;
  content: string;
  featuredImage: string;
  author: {
    name: string;
    avatar: string;
  };
}

async function getBlogPost(slug: string): Promise<BlogPost> {
  const response = await fetch(`https://api.example.com/blog/${slug}`);
  return response.json();
}

export default async function BlogPostPage({ params }: { params: { slug: string } }) {
  const post = await getBlogPost(params.slug);

  return (
    <article className="max-w-4xl mx-auto px-4 py-8">
      {/* Featured image from external source */}
      <div className="relative aspect-[16/9] bg-gray-100 rounded-lg overflow-hidden mb-8">
        <Image
          src={post.featuredImage}
          alt={post.title}
          fill
          priority
          quality={90}
          sizes="100vw"
          className="object-cover"
        />
      </div>

      {/* Author avatar from external source */}
      <div className="flex items-center space-x-4 mb-8">
        <div className="relative w-12 h-12 bg-gray-200 rounded-full overflow-hidden">
          <Image
            src={post.author.avatar}
            alt={post.author.name}
            fill
            sizes="48px"
            className="object-cover"
          />
        </div>
        <div>
          <p className="font-semibold text-gray-900">{post.author.name}</p>
          <p className="text-sm text-gray-600">Author</p>
        </div>
      </div>

      {/* Blog content */}
      <h1 className="text-4xl font-bold text-gray-900 mb-6">{post.title}</h1>
      <div className="prose prose-lg max-w-none">
        {post.content}
      </div>
    </article>
  );
}
```

### Remote Image Configuration

Configure remote image domains in `next.config.js`:

```javascript
// next.config.js
/** @type {import('next').NextConfig} */
const nextConfig = {
  images: {
    remotePatterns: [
      {
        protocol: 'https',
        hostname: 'api.example.com',
        port: '',
        pathname: '/images/**',
      },
      {
        protocol: 'https',
        hostname: 'cdn.example.com',
        port: '',
        pathname: '/uploads/**',
      },
      {
        protocol: 'https',
        hostname: '**.example.com', // Wildcard for subdomains
      },
    ],
    formats: ['image/avif', 'image/webp'],
    deviceSizes: [640, 750, 828, 1080, 1200, 1920, 2048, 3840],
    imageSizes: [16, 32, 48, 64, 96, 128, 256, 384],
  },
};

module.exports = nextConfig;
```

## 9.2 Font Optimization with next/font

Fonts significantly impact performance and perceived speed. Next.js's `next/font` automatically optimizes fonts (including custom fonts) and removes external network requests for improved privacy and performance.

### Using Google Fonts

Optimize Google Fonts with automatic self-hosting:

```tsx
// app/layout.tsx
import { Inter, Poppins, Space_Grotesk } from 'next/font/google';
import './globals.css';

// Configure Inter font with subsets
const inter = Inter({
  subsets: ['latin'],
  variable: '--font-inter',
  display: 'swap',
});

// Configure Poppins font with weights
const poppins = Poppins({
  weight: ['400', '500', '600', '700'],
  subsets: ['latin'],
  variable: '--font-poppins',
  display: 'swap',
});

// Configure Space Grotesk for headings
const spaceGrotesk = Space_Grotesk({
  weight: ['400', '500', '600', '700'],
  subsets: ['latin'],
  variable: '--font-space-grotesk',
  display: 'swap',
});

export const metadata = {
  title: 'Next.js Font Optimization',
  description: 'Learn about font optimization in Next.js',
};

export default function RootLayout({
  children,
}: {
  children: React.ReactNode;
}) {
  return (
    <html lang="en">
      <body className={`${inter.variable} ${poppins.variable} ${spaceGrotesk.variable} font-sans antialiased`}>
        {children}
      </body>
    </html>
  );
}
```

### Using Custom Fonts

Use custom local fonts with automatic optimization:

```tsx
// app/layout.tsx
import localFont from 'next/font/local';

// Load custom font files from local directory
const customFont = localFont({
  src: [
    {
      path: '../fonts/custom-regular.woff2',
      weight: '400',
      style: 'normal',
    },
    {
      path: '../fonts/custom-medium.woff2',
      weight: '500',
      style: 'normal',
    },
    {
      path: '../fonts/custom-bold.woff2',
      weight: '700',
      style: 'normal',
    },
  ],
  variable: '--font-custom',
  display: 'swap',
});

export default function RootLayout({
  children,
}: {
  children: React.ReactNode;
}) {
  return (
    <html lang="en">
      <body className={`${customFont.variable} font-sans antialiased`}>
        {children}
      </body>
    </html>
  );
}
```

### Font Variants and Weights

Use multiple font weights and styles:

```tsx
// components/Typography.tsx

export function TypographyExample() {
  return (
    <div className="container mx-auto px-4 py-8 space-y-8">
      {/* Heading with custom font weight */}
      <h1 className="text-5xl font-bold text-gray-900 mb-4">
        Next.js Font Optimization
      </h1>

      {/* Subheading */}
      <h2 className="text-2xl font-semibold text-gray-700 mb-2">
        Beautiful, Optimized Typography
      </h2>

      {/* Body text */}
      <p className="text-base text-gray-600 leading-relaxed max-w-3xl">
        Next.js automatically optimizes fonts by removing external network requests, 
        improving privacy and performance. Fonts are automatically self-hosted and 
        included in the build, ensuring fast load times.
      </p>

      {/* Different font weights */}
      <div className="space-y-2">
        <p className="font-light text-gray-600">Light weight text</p>
        <p className="font-normal text-gray-600">Normal weight text</p>
        <p className="font-medium text-gray-600">Medium weight text</p>
        <p className="font-semibold text-gray-600">Semibold weight text</p>
        <p className="font-bold text-gray-600">Bold weight text</p>
      </div>

      {/* Responsive typography */}
      <div className="text-xl sm:text-2xl md:text-3xl lg:text-4xl text-gray-900">
        Responsive typography that scales beautifully
      </div>
    </div>
  );
}
```

### Font Loading States

Manage font loading with fallback fonts:

```tsx
// components/FontLoadingExample.tsx
"use client";

import { useState, useEffect } from 'react';

export function FontLoadingExample() {
  const [fontsLoaded, setFontsLoaded] = useState(false);

  useEffect(() => {
    // Check if fonts have loaded
    if (document.fonts) {
      document.fonts.ready.then(() => {
        setFontsLoaded(true);
      });
    }
  }, []);

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="bg-white rounded-lg shadow-md p-6">
        {!fontsLoaded && (
          <div className="animate-pulse mb-4">
            <div className="h-8 bg-gray-200 rounded w-3/4 mb-2"></div>
            <div className="h-4 bg-gray-200 rounded"></div>
          </div>
        )}
        
        <h1 className={`text-3xl font-bold text-gray-900 ${!fontsLoaded ? 'opacity-0' : 'opacity-100'} transition-opacity duration-300`}>
          {fontsLoaded ? 'Fonts Loaded!' : 'Loading Fonts...'}
        </h1>
        
        <p className={`text-gray-600 mt-4 ${!fontsLoaded ? 'opacity-0' : 'opacity-100'} transition-opacity duration-300`}>
          This text displays with optimized fonts that are self-hosted and loaded instantly.
        </p>
      </div>
    </div>
  );
}
```

## 9.3 Script Optimization

Scripts, especially third-party scripts, can significantly impact performance. Next.js provides the `next/script` component to optimize script loading with strategies like lazy loading, deferment, and more.

### Basic Script Loading

Load scripts with different strategies:

```tsx
// app/layout.tsx
import Script from 'next/script';

export default function RootLayout({
  children,
}: {
  children: React.ReactNode;
}) {
  return (
    <html lang="en">
      <body>
        {children}

        {/* Google Analytics - Load after page is interactive */}
        <Script
          src="https://www.googletagmanager.com/gtag/js?id=GA_MEASUREMENT_ID"
          strategy="afterInteractive"
        />

        <Script id="google-analytics" strategy="afterInteractive">
          {`
            window.dataLayer = window.dataLayer || [];
            function gtag(){dataLayer.push(arguments);}
            gtag('js', new Date());
            gtag('config', 'GA_MEASUREMENT_ID');
          `}
        </Script>

        {/* Hotjar - Load lazily */}
        <Script
          src="https://static.hotjar.com/c/hotjar-12345.js"
          strategy="lazyOnload"
        />
      </body>
    </html>
  );
}
```

### Inline Scripts

Use inline scripts for critical functionality:

```tsx
// components/InlineScriptExample.tsx
import Script from 'next/script';

export function InlineScriptExample() {
  return (
    <div className="container mx-auto px-4 py-8">
      {/* Inline script for immediate execution */}
      <Script id="inline-script" strategy="beforeInteractive">
        {`
          // This script runs before the page is interactive
          console.log('Inline script executed');
          
          // Set up global configuration
          window.appConfig = {
            apiUrl: 'https://api.example.com',
            version: '1.0.0',
          };
        `}
      </Script>

      {/* Application content */}
      <div className="bg-white rounded-lg shadow-md p-6">
        <h1 className="text-2xl font-bold text-gray-900 mb-4">
          Inline Script Example
        </h1>
        <p className="text-gray-600">
          Check the console to see when the inline script executed.
        </p>
      </div>
    </div>
  );
}
```

### Third-Party Integration

Integrate popular third-party services:

```tsx
// components/ThirdPartyIntegrations.tsx
import Script from 'next/script';

export function ThirdPartyIntegrations() {
  return (
    <>
      {/* Stripe.js */}
      <Script
        src="https://js.stripe.com/v3/"
        strategy="lazyOnload"
        onLoad={() => {
          console.log('Stripe.js loaded');
          // Initialize Stripe
          window.stripe = Stripe('pk_test_12345');
        }}
      />

      {/* Intercom */}
      <Script
        src="https://widget.intercom.io/widget/ABC123"
        strategy="lazyOnload"
      />

      {/* Facebook Pixel */}
      <Script
        id="facebook-pixel"
        strategy="afterInteractive"
      >
        {`
          !function(f,b,e,v,n,t,s)
          {if(f.fbq)return;n=f.fbq=function(){n.callMethod?
          n.callMethod.apply(n,arguments):n.queue.push(arguments)};
          if(!f._fbq)f._fbq=n;n.push=n;n.loaded=!0;n.version='2.0';
          n.queue=[];t=b.createElement(e);t.async=!0;
          t.src=v;s=b.getElementsByTagName(e)[0];
          s.parentNode.insertBefore(t,s)}(window, document,'script',
          'https://connect.facebook.net/en_US/fbevents.js');
          fbq('init', '123456789'); 
          fbq('track', 'PageView');
        `}
      </Script>

      {/* NoScript fallback for Facebook Pixel */}
      <noscript>
        <img
          height="1"
          width="1"
          src="https://www.facebook.com/tr?id=123456789&ev=PageView&noscript=1"
          alt=""
        />
      </noscript>
    </>
  );
}
```

## 9.4 Automatic Code Splitting

Next.js automatically splits your code into smaller bundles, loading only the JavaScript needed for the current page. This significantly reduces initial load times and improves performance.

### Route-Based Code Splitting

Each page is automatically split into its own bundle:

```tsx
// app/dashboard/page.tsx - This file is automatically code-split
export default function DashboardPage() {
  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Dashboard</h1>
      <p className="text-gray-600">
        This page is automatically code-split by Next.js.
      </p>
    </div>
  );
}
```

### Dynamic Imports for Components

Load components dynamically when needed:

```tsx
// app/page.tsx
import dynamic from 'next/dynamic';

// Lazy load heavy components
const HeavyChart = dynamic(
  () => import('@/components/HeavyChart'),
  {
    loading: () => <div className="animate-pulse h-64 bg-gray-200 rounded" />,
    ssr: false, // Disable server-side rendering for client-only components
  }
);

const ComplexForm = dynamic(
  () => import('@/components/ComplexForm'),
  {
    loading: () => <div className="animate-pulse h-96 bg-gray-200 rounded" />,
  }
);

export default function HomePage() {
  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">
        Dynamic Imports
      </h1>

      <div className="grid grid-cols-1 lg:grid-cols-2 gap-8">
        {/* Chart loads only when scrolled into view */}
        <div>
          <h2 className="text-xl font-semibold mb-4">Analytics Chart</h2>
          <HeavyChart />
        </div>

        {/* Form loads only when user interacts */}
        <div>
          <h2 className="text-xl font-semibold mb-4">User Form</h2>
          <ComplexForm />
        </div>
      </div>
    </div>
  );
}
```

### Named Exports with Dynamic Imports

Import specific components dynamically:

```tsx
// components/Modal.tsx

// Named export 1
export function LoginModal() {
  return (
    <div className="bg-white rounded-lg shadow-lg p-6">
      <h2 className="text-2xl font-bold mb-4">Login</h2>
      {/* Login form */}
    </div>
  );
}

// Named export 2
export function SignupModal() {
  return (
    <div className="bg-white rounded-lg shadow-lg p-6">
      <h2 className="text-2xl font-bold mb-4">Sign Up</h2>
      {/* Signup form */}
    </div>
  );
}
```

```tsx
// app/auth/page.tsx
import dynamic from 'next/dynamic';

// Import specific named exports
const LoginModal = dynamic(
  () => import('@/components/Modal').then(mod => ({ default: mod.LoginModal })),
  { loading: () => <div>Loading login...</div> }
);

const SignupModal = dynamic(
  () => import('@/components/Modal').then(mod => ({ default: mod.SignupModal })),
  { loading: () => <div>Loading signup...</div> }
);

export default function AuthPage() {
  const [activeTab, setActiveTab] = useState<'login' | 'signup'>('login');

  return (
    <div className="container mx-auto px-4 py-8">
      <div className="flex space-x-4 mb-8">
        <button
          onClick={() => setActiveTab('login')}
          className={`px-4 py-2 rounded ${activeTab === 'login' ? 'bg-blue-600 text-white' : 'bg-gray-200'}`}
        >
          Login
        </button>
        <button
          onClick={() => setActiveTab('signup')}
          className={`px-4 py-2 rounded ${activeTab === 'signup' ? 'bg-blue-600 text-white' : 'bg-gray-200'}`}
        >
          Sign Up
        </button>
      </div>

      {activeTab === 'login' ? <LoginModal /> : <SignupModal />}
    </div>
  );
}
```

### Conditional Component Loading

Load components based on conditions:

```tsx
// app/admin/page.tsx
"use client";

import dynamic from 'next/dynamic';
import { useState } from 'react';

// Lazy load admin panels
const UserManagement = dynamic(
  () => import('@/components/admin/UserManagement'),
  { loading: () => <div className="animate-pulse h-64 bg-gray-200 rounded" />
);
const AnalyticsPanel = dynamic(
  () => import('@/components/admin/AnalyticsPanel'),
  { loading: () => <div className="animate-pulse h-64 bg-gray-200 rounded" />
);
const SettingsPanel = dynamic(
  () => import('@/components/admin/SettingsPanel'),
  { loading: () => <div className="animate-pulse h-64 bg-gray-200 rounded" />
);

export default function AdminPage() {
  const [activePanel, setActivePanel] = useState<'users' | 'analytics' | 'settings'>('users');

  return (
    <div className="container mx-auto px-4 py-8">
      <h1 className="text-3xl font-bold text-gray-900 mb-8">Admin Dashboard</h1>

      <div className="flex space-x-4 mb-8">
        <button
          onClick={() => setActivePanel('users')}
          className={`px-4 py-2 rounded ${activePanel === 'users' ? 'bg-blue-600 text-white' : 'bg-gray-200'}`}
        >
          Users
        </button>
        <button
          onClick={() => setActivePanel('analytics')}
          className={`px-4 py-2 rounded ${activePanel === 'analytics' ? 'bg-blue-600 text-white' : 'bg-gray-200'}`}
        >
          Analytics
        </button>
        <button
          onClick={() => setActivePanel('settings')}
          className={`px-4 py-2 rounded ${activePanel === 'settings' ? 'bg-blue-600 text-white' : 'bg-gray-200'}`}
        >
          Settings
        </button>
      </div>

      {/* Only load the active panel */}
      {activePanel === 'users' && <UserManagement />}
      {activePanel === 'analytics' && <AnalyticsPanel />}
      {activePanel === 'settings' && <SettingsPanel />}
    </div>
  );
}
```

## 9.5 Bundle Analysis

Analyzing your bundle helps identify large dependencies and optimization opportunities. Next.js provides built-in tools for bundle analysis.

### Using @next/bundle-analyzer

Set up bundle analysis:

```bash
npm install @next/bundle-analyzer
```

```javascript
// next.config.js
const withBundleAnalyzer = require('@next/bundle-analyzer')({
  enabled: process.env.ANALYZE === 'true',
});

/** @type {import('next').NextConfig} */
const nextConfig = {
  // ... your config
};

module.exports = withBundleAnalyzer(nextConfig);
```

**Run bundle analysis:**

```bash
ANALYZE=true npm run build
```

### Webpack Bundle Analyzer

Alternative bundle analyzer:

```bash
npm install -D @webpack-bundle-analyzer webpack-bundle-analyzer
```

```javascript
// next.config.js
const BundleAnalyzer = require('@next/bundle-analyzer');

/** @type {import('next').NextConfig} */
const nextConfig = {
  webpack(config, { isServer }) {
    if (process.env.ANALYZE === 'true') {
      config.plugins.push(
        new BundleAnalyzer.BundleAnalyzerPlugin({
          analyzerMode: 'server',
          analyzerPort: isServer ? 8888 : 8889,
          openAnalyzer: true,
        })
      );
    }
    return config;
  },
};

module.exports = nextConfig;
```

### Analyzing Bundle Size

Create a script to analyze bundle size:

```javascript
// scripts/analyze-bundle.js
const { execSync } = require('child_process');
const fs = require('fs');
const path = require('path');

function analyzeBundle() {
  console.log('🔍 Analyzing bundle size...');
  
  // Run Next.js build
  execSync('npm run build', { stdio: 'inherit' });
  
  // Read build output
  const buildDir = path.join(__dirname, '..', '.next');
  const staticDir = path.join(buildDir, 'static');
  
  if (fs.existsSync(staticDir)) {
    function getDirectorySize(dirPath) {
      let size = 0;
      const files = fs.readdirSync(dirPath);
      
      files.forEach(file => {
        const filePath = path.join(dirPath, file);
        const stats = fs.statSync(filePath);
        
        if (stats.isDirectory()) {
          size += getDirectorySize(filePath);
        } else {
          size += stats.size;
        }
      });
      
      return size;
    }
    
    const totalSize = getDirectorySize(staticDir);
    const sizeInMB = (totalSize / (1024 * 1024)).toFixed(2);
    
    console.log(`\n📦 Total bundle size: ${sizeInMB} MB`);
    
    // Log individual files
    function logFiles(dirPath, depth = 0) {
      const files = fs.readdirSync(dirPath);
      
      files.forEach(file => {
        const filePath = path.join(dirPath, file);
        const stats = fs.statSync(filePath);
        const indent = '  '.repeat(depth);
        
        if (stats.isDirectory()) {
          console.log(`${indent}📁 ${file}/`);
          logFiles(filePath, depth + 1);
        } else {
          const sizeInKB = (stats.size / 1024).toFixed(2);
          console.log(`${indent}📄 ${file} (${sizeInKB} KB)`);
        }
      });
    }
    
    console.log('\n📋 Bundle contents:');
    logFiles(staticDir);
  }
}

analyzeBundle();
```

**Add to package.json:**

```json
{
  "scripts": {
    "analyze": "node scripts/analyze-bundle.js"
  }
}
```

## 9.6 Performance Monitoring

Monitoring performance helps identify bottlenecks and track improvements over time.

### Core Web Vitals Monitoring

Track Core Web Vitals:

```tsx
// components/WebVitals.tsx
"use client";

import { useEffect } from 'react';

export function WebVitals() {
  useEffect(() => {
    if (typeof window !== 'undefined' && 'PerformanceObserver' in window) {
      // Largest Contentful Paint (LCP)
      const lcpObserver = new PerformanceObserver((list) => {
        const entries = list.getEntries();
        const lastEntry = entries[entries.length - 1];
        console.log('LCP:', lastEntry.startTime);
      });
      lcpObserver.observe({ entryTypes: ['largest-contentful-paint'] });

      // First Input Delay (FID)
      const fidObserver = new PerformanceObserver((list) => {
        for (const entry of list.getEntries()) {
          console.log('FID:', entry.processingStart - entry.startTime);
        }
      });
      fidObserver.observe({ entryTypes: ['first-input'] });

      // Cumulative Layout Shift (CLS)
      let clsValue = 0;
      const clsObserver = new PerformanceObserver((list) => {
        for (const entry of list.getEntries()) {
          if (!entry.hadRecentInput) {
            clsValue += entry.value;
            console.log('CLS:', clsValue);
          }
        }
      });
      clsObserver.observe({ entryTypes: ['layout-shift'] });
    }
  }, []);

  return null;
}
```

### Performance Metrics Dashboard

Create a performance metrics dashboard:

```tsx
// components/PerformanceDashboard.tsx
"use client";

import { useState, useEffect } from 'react';

interface PerformanceMetrics {
  lcp?: number;
  fid?: number;
  cls?: number;
  fcp?: number;
  ttfb?: number;
}

export function PerformanceDashboard() {
  const [metrics, setMetrics] = useState<PerformanceMetrics>({});
  const [loading, setLoading] = useState(true);

  useEffect(() => {
    if (typeof window !== 'undefined' && 'PerformanceObserver' in window) {
      const newMetrics: PerformanceMetrics = {};

      // LCP
      const lcpObserver = new PerformanceObserver((list) => {
        const entries = list.getEntries();
        const lastEntry = entries[entries.length - 1];
        newMetrics.lcp = lastEntry.startTime;
        setMetrics({ ...newMetrics });
      });
      lcpObserver.observe({ entryTypes: ['largest-contentful-paint'] });

      // FID
      const fidObserver = new PerformanceObserver((list) => {
        for (const entry of list.getEntries()) {
          newMetrics.fid = entry.processingStart - entry.startTime;
          setMetrics({ ...newMetrics });
        }
      });
      fidObserver.observe({ entryTypes: ['first-input'] });

      // CLS
      let clsValue = 0;
      const clsObserver = new PerformanceObserver((list) => {
        for (const entry of list.getEntries()) {
          if (!entry.hadRecentInput) {
            clsValue += entry.value;
            newMetrics.cls = clsValue;
            setMetrics({ ...newMetrics });
          }
        }
      });
      clsObserver.observe({ entryTypes: ['layout-shift'] });

      // FCP
      const fcpObserver = new PerformanceObserver((list) => {
        for (const entry of list.getEntries()) {
          if (entry.name === 'first-contentful-paint') {
            newMetrics.fcp = entry.startTime;
            setMetrics({ ...newMetrics });
          }
        }
      });
      fcpObserver.observe({ entryTypes: ['paint'] });

      // TTFB
      if (performance.timing) {
        newMetrics.ttfb = performance.timing.responseStart - performance.timing.navigationStart;
        setMetrics({ ...newMetrics });
      }

      setTimeout(() => setLoading(false), 1000);
    }
  }, []);

  const getScore = (value: number | undefined, thresholds: { good: number; needsImprovement: number }) => {
    if (!value) return { score: 'N/A', color: 'gray' };
    if (value <= thresholds.good) return { score: 'Good', color: 'green' };
    if (value <= thresholds.needsImprovement) return { score: 'Needs Improvement', color: 'yellow' };
    return { score: 'Poor', color: 'red' };
  };

  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-bold text-gray-900 mb-6">Performance Metrics</h2>

      {loading ? (
        <div className="animate-pulse">
          <div className="h-6 bg-gray-200 rounded w-1/2 mb-4"></div>
          <div className="space-y-3">
            {[1, 2, 3, 4].map((i) => (
              <div key={i} className="h-12 bg-gray-200 rounded"></div>
            ))}
          </div>
        </div>
      ) : (
        <div className="space-y-4">
          <MetricCard
            label="Largest Contentful Paint (LCP)"
            value={metrics.lcp}
            unit="ms"
            thresholds={{ good: 2500, needsImprovement: 4000 }}
            description="Time it takes for the main content to load"
          />
          <MetricCard
            label="First Input Delay (FID)"
            value={metrics.fid}
            unit="ms"
            thresholds={{ good: 100, needsImprovement: 300 }}
            description="Time from user's first interaction to browser response"
          />
          <MetricCard
            label="Cumulative Layout Shift (CLS)"
            value={metrics.cls}
            unit=""
            thresholds={{ good: 0.1, needsImprovement: 0.25 }}
            description="Amount of unexpected layout shift"
          />
          <MetricCard
            label="First Contentful Paint (FCP)"
            value={metrics.fcp}
            unit="ms"
            thresholds={{ good: 1800, needsImprovement: 3000 }}
            description="Time when first content is painted"
          />
          <MetricCard
            label="Time to First Byte (TTFB)"
            value={metrics.ttfb}
            unit="ms"
            thresholds={{ good: 800, needsImprovement: 1800 }}
            description="Time from request to first byte of response"
          />
        </div>
      )}
    </div>
  );
}

function MetricCard({
  label,
  value,
  unit,
  thresholds,
  description,
}: {
  label: string;
  value: number | undefined;
  unit: string;
  thresholds: { good: number; needsImprovement: number };
  description: string;
}) {
  const { score, color } = getScore(value, thresholds);
  const colorClasses = {
    green: 'bg-green-100 text-green-800',
    yellow: 'bg-yellow-100 text-yellow-800',
    red: 'bg-red-100 text-red-800',
    gray: 'bg-gray-100 text-gray-800',
  };

  return (
    <div className="border border-gray-200 rounded-lg p-4">
      <div className="flex justify-between items-start mb-2">
        <div>
          <h3 className="font-semibold text-gray-900">{label}</h3>
          <p className="text-sm text-gray-600">{description}</p>
        </div>
        <span className={`px-2 py-1 rounded-full text-xs font-medium ${colorClasses[color as keyof typeof colorClasses]}`}>
          {score}
        </span>
      </div>
      <div className="text-2xl font-bold text-gray-900">
        {value !== undefined ? `${value.toFixed(0)}${unit}` : 'N/A'}
      </div>
    </div>
  );
}
```

### Custom Performance Monitoring

Implement custom performance tracking:

```tsx
// lib/performance.ts

export class PerformanceMonitor {
  private marks: Map<string, number> = new Map();
  private measures: Map<string, number[]> = new Map();

  // Mark a point in time
  mark(name: string) {
    if (typeof window !== 'undefined') {
      performance.mark(name);
      this.marks.set(name, performance.now());
    }
  }

  // Measure time between marks
  measure(name: string, startMark: string, endMark?: string) {
    if (typeof window !== 'undefined') {
      try {
        performance.measure(name, startMark, endMark);
        const measure = performance.getEntriesByName(name)[0];
        
        if (!this.measures.has(name)) {
          this.measures.set(name, []);
        }
        
        this.measures.get(name)!.push(measure.duration);
        
        return measure.duration;
      } catch (error) {
        console.error('Performance measure error:', error);
        return 0;
      }
    }
    return 0;
  }

  // Get average duration for a measure
  getAverage(name: string): number {
    const durations = this.measures.get(name);
    if (!durations || durations.length === 0) return 0;
    
    const sum = durations.reduce((acc, val) => acc + val, 0);
    return sum / durations.length;
  }

  // Get statistics for a measure
  getStats(name: string) {
    const durations = this.measures.get(name);
    if (!durations || durations.length === 0) return null;
    
    const sorted = [...durations].sort((a, b) => a - b);
    const min = sorted[0];
    const max = sorted[sorted.length - 1];
    const median = sorted[Math.floor(sorted.length / 2)];
    const average = this.getAverage(name);
    
    return { min, max, median, average, count: sorted.length };
  }

  // Clear all marks and measures
  clear() {
    if (typeof window !== 'undefined') {
      performance.clearMarks();
      performance.clearMeasures();
      this.marks.clear();
      this.measures.clear();
    }
  }
}

// Global performance monitor instance
export const performanceMonitor = new PerformanceMonitor();
```

**Using the performance monitor:**

```tsx
// components/PerformanceTrackedComponent.tsx
"use client";

import { useEffect } from 'react';
import { performanceMonitor } from '@/lib/performance';

export function PerformanceTrackedComponent() {
  useEffect(() => {
    // Mark start of component mount
    performanceMonitor.mark('component-mount-start');
    
    // Simulate component initialization
    setTimeout(() => {
      performanceMonitor.mark('component-mount-end');
      const mountTime = performanceMonitor.measure('component-mount', 'component-mount-start', 'component-mount-end');
      console.log('Component mount time:', mountTime, 'ms');
    }, 100);

    // Track data fetch
    performanceMonitor.mark('data-fetch-start');
    fetchData().then(() => {
      performanceMonitor.mark('data-fetch-end');
      const fetchTime = performanceMonitor.measure('data-fetch', 'data-fetch-start', 'data-fetch-end');
      console.log('Data fetch time:', fetchTime, 'ms');
    });
  }, []);

  return (
    <div className="bg-white rounded-lg shadow-md p-6">
      <h2 className="text-xl font-bold text-gray-900 mb-4">
        Performance Tracked Component
      </h2>
      <p className="text-gray-600">
        This component tracks its performance metrics.
      </p>
    </div>
  );
}

async function fetchData() {
  // Simulate API call
  await new Promise(resolve => setTimeout(resolve, 500));
}
```

## Key Takeaways from Chapter 9

1. **Image Optimization**: Next.js's `next/image` component provides automatic optimization, responsive images, lazy loading, modern format support (AVIF, WebP), and blur-up placeholders for improved performance and user experience.

2. **Font Optimization**: `next/font` automatically optimizes fonts by removing external network requests, supporting both Google Fonts and custom local fonts with automatic self-hosting and configuration of font weights, styles, and loading strategies.

3. **Script Optimization**: `next/script` provides strategies like `beforeInteractive`, `afterInteractive`, and `lazyOnload` to optimize script loading, supporting both external and inline scripts for better performance.

4. **Automatic Code Splitting**: Next.js automatically splits code at the route level and supports dynamic imports with `dynamic()` for components, with loading states and SSR control for optimal bundle sizes.

5. **Bundle Analysis**: Use `@next/bundle-analyzer` or alternative tools to analyze bundle sizes, identify large dependencies, and optimize by removing unused code and splitting bundles appropriately.

6. **Performance Monitoring**: Track Core Web Vitals (LCP, FID, CLS) and implement custom performance monitoring with the Performance API to identify bottlenecks and track improvements over time.

## Coming Up Next

**Chapter 10: Advanced Data Fetching & Caching**

Now that you understand built-in optimizations, it's time to dive deeper into advanced data fetching and caching strategies. In Chapter 10, we'll explore Cache Components and the "use cache" directive, revalidation strategies, on-demand revalidation, tag-based cache invalidation, Incremental Static Regeneration (ISR), streaming with Suspense, and parallel and sequential data fetching patterns.